In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
os.chdir('/content/drive/MyDrive/')

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Input, Flatten, Dense, Activation, BatchNormalization, LeakyReLU, Conv2DTranspose, Layer, Reshape
#from keras.layers import add,AveragePooling2D
#from keras.regularizers import l2
from keras.models import Model
from keras.datasets import mnist
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
#from keras.utils import plot_model
#from keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
#from scipy.io import loadmat
#import h5py
#import glob
#import pandas as pd
import tensorflow as tf


In [11]:
#hyper parameter
latent_dim=100
image_size=256;
image_shape=(image_size,image_size,1)
gen_filters=[128,64,32,1]
gen_strides=[2, 2, 1, 1]
dis_filters=[32,64,128,256]
dis_strides=[2, 2, 2, 1]
kernel_size=5
strides=2
alpha=0.2
#batch_size=64
batch_size=8
dis_lr=2e-4
dis_decay=6e-8
dis_optimizer=RMSprop(lr=dis_lr, decay=dis_decay)
adv_lr=dis_lr*0.5
adv_decay=dis_decay*0.5
adv_optimizer=RMSprop(lr=adv_lr, decay=adv_decay)
save_intervals=500
train_steps=40000
log_print_steps=50
#test_size=16
#test_size=4
test_size=1

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
#load mnist data
#(x_train, _), (x_test, _)=mnist.load_data()
import cv2
from sklearn.model_selection import train_test_split
import glob
all_feature=[]
i=0
for item in glob.glob("/content/drive/MyDrive/retina_classification//0//*"):
    i+=1
    img=cv2.imread(item)
    img=cv2.resize(img,(256,256))
    #all_feature.append(img)
    if i%1000==0:
        print("[INFO] processed {}/25000".format(i))
#(x_train, _), (x_test, _)=train_test_split(all_feature,test_size=0.09
x_train=img
#print(x_train.shape)

x_train=x_train.reshape((-1, image_size, image_size, 1)).astype('float32')/255
print(x_train.shape)

(3, 256, 256, 1)


In [14]:
import os
os.makedirs('images', exist_ok = True)

In [15]:
def bn_relu(x):
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    return x


In [16]:
def build_generator(inputs, image_size=256):
    filters=gen_filters
    x=inputs
    image_resize=image_size//4 
    x=Dense(image_resize * image_resize * filters[0])(x)
    x=Reshape((image_resize, image_resize, filters[0]))(x)
    for strides, filter in zip(gen_strides, filters):
        x=bn_relu(x)
        x=Conv2DTranspose(filters=filter,
                          kernel_size=kernel_size,
                          padding='same',
                          strides=strides)(x)
    outputs=Activation('sigmoid', name='Sigmoid')(x)
    model=Model(inputs, outputs, name='generator')
    model.summary
   # plot_model(model, to_file='generator.png', show_shape=True)
    return model

In [17]:
def build_discriminator(inputs):
    x=inputs
    for strides, filter in zip(dis_strides, dis_filters):
        x=LeakyReLU(alpha=alpha)(x)
        x=Conv2D(filters=filter,
                          kernel_size=kernel_size,
                          padding='same',
                          strides=strides)(x)
    x=Flatten()(x)
    x=Dense(1)(x)
    outputs=Activation('sigmoid', name='Sigmoid')(x)
    model=Model(inputs, outputs, name='discriminator')
    return model

        

In [18]:
def build_and_train():
    dis_inputs=Input(shape=image_shape, name='dis_inputs')  #ye batch midim
    dis=build_discriminator(dis_inputs)
    dis.compile(loss='binary_crossentropy',
                optimizer=dis_optimizer,
                metrics=['acc']
                 )
    gen_inputs=Input(shape=(latent_dim,), name='gen_inputs')
    gen=build_generator(gen_inputs)
    dis.trainable= False
    adv_inputs= gen_inputs
    adv_outputs=dis(gen(gen_inputs))
    adv=Model(adv_inputs, adv_outputs, name='adversarial')
    adv.compile(loss='binary_crossentropy',
                optimizer=adv_optimizer,
                metrics=['acc'])
    #plot_model(adv, to_file='adversarial.png', show_shape=True)
    adv.summary
    models= gen, dis, adv

    train(models)


In [19]:
def train(models):
    gen, dis, adv =models
    m_train=x_train.shape[0]
    test_z= np.random.uniform(low=-1, high=+1, size=[test_size, latent_dim])
    for step in range(1, train_steps + 1):
        random_indices= np.random.randint(0, m_train, size=batch_size)
        real_images=x_train[random_indices]
        z=np.random.uniform(low=-1, high=+1, size=[batch_size, latent_dim])
        fake_images= gen.predict(z)
        fake_labels=np.zeros((batch_size, 1))
        real_labels=np.ones((batch_size, 1))
        dis_x=np.concatenate([real_images, fake_images])
        dis_y=np.concatenate([real_labels, fake_labels])
        dis.train_on_batch(dis_x, dis_y)
        loss, acc= dis.train_on_batch(dis_x, dis_y)
        log = f'step: {step} dis[loss:{loss}, acc:{acc}]'
        adv_x=np.random.uniform(low=-1, high=+1, size=[batch_size, latent_dim])
        adv_y=np.ones((batch_size, 1))
        adv.train_on_batch(adv_x, adv_y)
        loss, acc= adv.train_on_batch(adv_x, adv_y)
        log += f'adv[loss:{loss}, acc:{acc}]'
        print(log)
        
        if step % save_intervals == 0:
           plot_images(gen, test_z, step)


In [20]:
def plot_images(generator, noise_input, step, show=False):
    

    image_address='images'
    z=noise_input
    n_images=z.shape[0]
    rows=np.sqrt(n_images)
    plt.figure(figsize=(2,2))
    images=generator.predict(z)
    image_size=images.shape[1]
    for i in range(n_images):
        plt.subplot(rows, rows, i+1)
        plt.imshow(images[i].reshape((image_size, image_size)), cmap='hot')
        plt.axis('off')
    plt.savefig(os.path.join(image_address, f"(step).png"))
    if show:
       plt.show()
    else:
       plt.close('all')

In [ ]:
build_and_train()

step: 1 dis[loss:0.6247862577438354, acc:0.5]adv[loss:0.00024921935983002186, acc:1.0]
step: 2 dis[loss:1.251990556716919, acc:0.5]adv[loss:0.0023254286497831345, acc:1.0]
step: 3 dis[loss:0.5473417043685913, acc:0.8125]adv[loss:0.2712036073207855, acc:1.0]
step: 4 dis[loss:0.37664371728897095, acc:1.0]adv[loss:0.045120567083358765, acc:1.0]
step: 5 dis[loss:0.1992185413837433, acc:1.0]adv[loss:0.005991633981466293, acc:1.0]
step: 6 dis[loss:0.08189623802900314, acc:1.0]adv[loss:0.0032495572231709957, acc:1.0]
step: 7 dis[loss:0.07636971771717072, acc:1.0]adv[loss:0.002450124593451619, acc:1.0]
step: 8 dis[loss:0.23911546170711517, acc:1.0]adv[loss:0.003954588435590267, acc:1.0]
step: 9 dis[loss:0.025605211034417152, acc:1.0]adv[loss:0.0009281778475269675, acc:1.0]
step: 10 dis[loss:0.012447157874703407, acc:1.0]adv[loss:0.0008723651990294456, acc:1.0]
step: 11 dis[loss:0.011269161477684975, acc:1.0]adv[loss:0.00012720157974399626, acc:1.0]
step: 12 dis[loss:0.006738197989761829, acc:1

In [ ]:
def test_generator(generator):
    noise_inputs=np.random.uniform(-1.0, 1.0, size=[16,100])
    plot_images(generator,
                noise_input=noise_input,
                show=True,
                model_name="test_outputs")
